<a href="https://colab.research.google.com/github/alicedambroz/Discovery-Farms-curve-number/blob/main/Discovery_Farms_Curve_Number.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Use this code to estimate Curve Number from flow and precipitation data**

Data files contain 3 spreadsheets per site. Those include Precipitation, Runoff and Events.

1.   **Precipitation** = minute interval rainfall data in inches and milimiters.
2.   **Runoff** = minute interval discharge in cfs was transformed to volume in milimiters.
3.   **Events** = start and end date and time of each event, as separated in Discovery Farms original files.

Sites under analysis here are: Redwood South (no-till without cover crops), Redwood North (no-till with cover crops after 2021) and McLeod (conventional tillage).

*Calculations are run on the metric system.*


**To run code:**
*   Get data files and save them as Google Sheets on Google Drive. Use URL code in sites_config

In [ ]:
from google.colab import auth, files
from google.auth import default
import gspread
import pandas as pd
import numpy as np

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

#Batch run for RWS, RWN and MC

In [ ]:
# OPEN DATA
# From Google Sheets, paste URL from each site's file

sites_config = {
    'MC1':  '152H5O0QNSPoVYDgHuSD8Tq7C4rWKppXwhh8TSeP1DsY',   #spreadsheet ID here
    'RW1N': '1wbd9KU4-aVSgwU39b-P8cs9UeREUWwNGzooJYV5GLRk',  #spreadsheet ID here
    'RW1S': '10kwU-3ZJ1lkK-5dymXfjpI-Zz2tE-1cqnVVSo-Y_SBE' #spreadsheet ID here
}

# PROCESS DATA
# Functions to configure data


def process_site_data(site_name, sheet_id):
    print(f"Processing Site: {site_name}...")

    try:
        spreadsheet = gc.open_by_key(sheet_id)
    except Exception as e:
        print(f"Error opening sheet for {site_name}: {e}")
        return pd.DataFrame()

    # Get data from each spreadsheet
    sheet_p = spreadsheet.worksheet("Precipitation")
    rows_p = sheet_p.get_all_values()
    df_p = pd.DataFrame(rows_p[1:], columns=rows_p[0])

    sheet_q = spreadsheet.worksheet("Runoff")
    rows_q = sheet_q.get_all_values()
    df_q = pd.DataFrame(rows_q[1:], columns=rows_q[0])

    sheet_ev = spreadsheet.worksheet("Events")
    rows_ev = sheet_ev.get_all_values()
    df_events_meta = pd.DataFrame(rows_ev[1:], columns=rows_ev[0])

    # Clean and transform data
    # Precipitation
    df_p['Rainfall_mm'] = pd.to_numeric(df_p['Rainfall_mm'], errors='coerce').fillna(0)
    df_p['Date'] = pd.to_datetime(df_p['Date'], format='mixed', dayfirst=False)

    # Runoff
    df_q['Volume_mm'] = pd.to_numeric(df_q['Volume_mm'], errors='coerce').fillna(0)
    df_q['Event'] = pd.to_numeric(df_q['Event'], errors='coerce').fillna(0)
    df_q['Date'] = pd.to_datetime(df_q['Date'], format='mixed', dayfirst=False)

    # Events
    df_events_meta['Event'] = pd.to_numeric(df_events_meta['Event'], errors='coerce')
    df_events_meta['Start'] = pd.to_datetime(df_events_meta['Start'], format='mixed', dayfirst=False)
    df_events_meta['End'] = pd.to_datetime(df_events_meta['End'], format='mixed', dayfirst=False)

    # Frozen and nonfrozen events
    # Clean 'Frozen' column and convert to string
    if 'Frozen' in df_events_meta.columns:
        df_events_meta['Frozen'] = df_events_meta['Frozen'].astype(str)
    else:
        df_events_meta['Frozen'] = 'Unknown'

    df_events_meta = df_events_meta[df_events_meta['Event'] > 0].copy()

    # Creates master timeline of 1-minute interval
    min_date = min(df_p['Date'].min(), df_q['Date'].min())
    max_date = max(df_p['Date'].max(), df_q['Date'].max())
    time_index = pd.date_range(start=min_date, end=max_date, freq='1min')
    df_master = pd.DataFrame({'Date': time_index})

    # Merge precipitation and runoff data with master timeline dataframe
    df_master = df_master.merge(df_q[['Date', 'Volume_mm']], on='Date', how='left')
    df_master = df_master.merge(df_p[['Date', 'Rainfall_mm']], on='Date', how='left')
    df_master['Volume_mm'] = df_master['Volume_mm'].fillna(0)
    df_master['Rainfall_mm'] = df_master['Rainfall_mm'].fillna(0)

    # Extract events
    site_events_list = []

    for _, ev in df_events_meta.iterrows():
        event_id = ev['Event']
        start_flow = ev['Start']
        end_flow = ev['End']
        frozen_status = ev['Frozen'] #'Frozen' status

        if pd.isna(start_flow) or pd.isna(end_flow):
            continue

        start_plot = start_flow - pd.Timedelta(hours=6)

        mask = (df_master['Date'] >= start_plot) & (df_master['Date'] <= end_flow)
        df_event_slice = df_master.loc[mask].copy()

        df_event_slice['Event'] = event_id
        df_event_slice['Site'] = site_name
        df_event_slice['Frozen'] = frozen_status

        site_events_list.append(df_event_slice)

    if site_events_list:
        return pd.concat(site_events_list, ignore_index=True)
    else:
        return pd.DataFrame()

# FINAL DATAFRAME
# Get merged and organized data ready for plotting and for calculating CN

all_sites_data = []

# Loop through the site dictionary defined previously
for site, sheet_id in sites_config.items():
    df_site = process_site_data(site, sheet_id)
    if not df_site.empty:
        all_sites_data.append(df_site)

# Concatenate all sites data into one final dataframe
if all_sites_data:
    df_final = pd.concat(all_sites_data, ignore_index=True)

    # Preview
    print("\nProcessing Complete!")
    print(df_final.head())
    print(f"Total rows: {len(df_final)}")

    # Save and download
    df_final.to_csv('All_Sites_Events_Plotting.csv', index=False)
    files.download('All_Sites_Events_Plotting.csv')
else:
    print("No data processed.")

# This dataframe can now be used in R for plotting each hydrograph!

Processing Site: MC1...
Processing Site: RW1N...
Processing Site: RW1S...

Processing Complete!
                 Date  Volume_mm  Rainfall_mm  Event Site  Frozen
0 2017-02-15 07:14:00        0.0          0.0      1  MC1  frozen
1 2017-02-15 07:15:00        0.0          0.0      1  MC1  frozen
2 2017-02-15 07:16:00        0.0          0.0      1  MC1  frozen
3 2017-02-15 07:17:00        0.0          0.0      1  MC1  frozen
4 2017-02-15 07:18:00        0.0          0.0      1  MC1  frozen
Total rows: 273363


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# CALCULATE S AND CN
# Final dataframe from previous chunk can now be used to calculate S and CN


print("Calculating Curve Numbers...")

# GROUP INFO
# Group information from each event and each site to calculate precipitation and runoff volume totals, maintaining 'Frozen' status
summary = df_final.groupby(['Site', 'Event', 'Frozen']).agg(
    Start_Time=('Date', 'min'),
    End_Time=('Date', 'max'),
    Total_Rainfall_mm=('Rainfall_mm', 'sum'),
    Total_Runoff_mm=('Volume_mm', 'sum')
).reset_index()

# S AND CN FUNCTIONS
def calculate_S(P, Q):
    # Formula to estimate S:
    # S = 5 * (P + 2Q - sqrt(4Q^2 + 5PQ))
          # P and Q are in mm.
          # For P >= Q! If Q > P, some error...

    if Q <= 0:
        return np.nan # If runoff is 0, we can assume all precipitation was stored
    if Q >= P:
        return 0 # If runoff is greater than precipitation, we can assume saturation, S=0, or some type of data error

    term1 = P + (2 * Q)
    term2 = np.sqrt((4 * (Q**2)) + (5 * P * Q))
    S = 5 * (term1 - term2)
    return S

def calculate_CN(S):
    if pd.isna(S):
        return np.nan
    # Formula to estimate CN in the metric system:
    # CN = 25400 / (S + 254)
    return 25400 / (S + 254)

# CALCULATE S AND CN
summary['S_mm'] = summary.apply(
    lambda row: calculate_S(row['Total_Rainfall_mm'], row['Total_Runoff_mm']), axis=1
)

summary['CN'] = summary['S_mm'].apply(calculate_CN)

# To clean and round decimals
summary['Total_Rainfall_mm'] = summary['Total_Rainfall_mm'].round(4)
summary['Total_Runoff_mm'] = summary['Total_Runoff_mm'].round(4)
summary['S_mm'] = summary['S_mm'].round(4)
summary['CN'] = summary['CN'].round(4)

# Sort data per site and event number
summary = summary.sort_values(by=['Site', 'Event'])

# Check output
print("\nCalculation Complete! Preview:")
print(summary.head())

# Save and download
summary.to_csv('Event_Summary_CN_S.csv', index=False)
files.download('Event_Summary_CN_S.csv')

Calculating Curve Numbers...

Calculation Complete! Preview:
  Site  Event     Frozen          Start_Time            End_Time  \
0  MC1      1     frozen 2017-02-15 07:14:00 2017-02-18 17:16:00   
1  MC1      2  nonfrozen 2017-04-14 23:00:00 2017-04-15 08:23:00   
2  MC1      3  nonfrozen 2017-04-19 09:24:00 2017-04-19 20:00:00   
3  MC1      4  nonfrozen 2017-04-30 15:00:00 2017-05-01 09:30:00   
4  MC1      5  nonfrozen 2017-05-16 19:46:00 2017-05-18 00:10:00   

   Total_Rainfall_mm  Total_Runoff_mm      S_mm        CN  
0              0.000           1.4416    0.0000  100.0000  
1             25.146           0.0081  120.7776   67.7735  
2             20.828           0.0136   98.3187   72.0938  
3             31.750           0.0391  146.6791   63.3924  
4             31.750           0.6395  114.3631   68.9537  


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#Tests: DO NOT RUN

##RW NORTH SITE

##Get rainfall data

In [ ]:
id = '1LZIpzGVfSbKe9J8NH3sRRE_n26yi4HeRQdapKdx43dQ' #id da planilha do google sheets - retirado do url
spreadsheet = gc.open_by_key(id) #para abrir spreadsheet pela chave
sheet = spreadsheet.worksheet("Precipitation") #nome da planilha a ser trabalhada
rows = sheet.get_all_values()
df_p_n = pd.DataFrame(rows[1:]) #transforma em DataFrame, a partir da linha 1 (linha 0 = cabeçalho)
df_p_n.columns = rows[0] #para transformar a primeira linha

In [ ]:
df_p_n.info()

In [ ]:
df_p_n['Rainfall_mm'] = pd.to_numeric(df_p_n['Rainfall_mm'], errors='coerce') #transformar object em float
df_p_n['Rainfall_mm'].fillna(0, inplace=True) #preenche NA com 0
df_p_n['Date'] = pd.to_datetime(df_p_n['Date'], format="%m/%d/%Y %H:%M:%S") #converte string para datetime

In [ ]:
df_p_n.info()

##Get runoff data

In [ ]:
id = '1LZIpzGVfSbKe9J8NH3sRRE_n26yi4HeRQdapKdx43dQ' #id da planilha do google sheets - retirado do url
spreadsheet = gc.open_by_key(id) #para abrir spreadsheet pela chave
sheet = spreadsheet.worksheet("Runoff") #nome da planilha a ser trabalhada
rows = sheet.get_all_values()
df_q_n = pd.DataFrame(rows[1:]) #transforma em DataFrame, a partir da linha 1 (linha 0 = cabeçalho)
df_q_n.columns = rows[0] #para transformar a primeira linha

In [ ]:
df_q_n.info()

In [ ]:
df_q_n['Volume_mm'] = pd.to_numeric(df_q_n['Volume_mm'], errors='coerce') #transformar object em float
df_q_n['Volume_mm'].fillna(0, inplace=True) #preenche NA com 0
df_q_n['Event'] = pd.to_numeric(df_q_n['Event'], errors='coerce') #transformar object em float
df_q_n['Event'].fillna(0, inplace=True) #preenche NA com 0
df_q_n['Date'] = pd.to_datetime(df_q_n['Date'], format="%m/%d/%Y %H:%M:%S") #converte string para datetime

In [ ]:
df_q_n.info()

##Get start and end of each event

In [ ]:
events_time_n = (
    df_q_n
    .groupby('Event')['Date']
    .agg(start_time='min', end_time='max')
    .reset_index()
)

In [ ]:
events_time_n = events_time_n[events_time_n['Event'] != 0]

In [ ]:
#Save
events_time_n.to_csv('RW1S_events_start_end.csv', index=True)

#Download
files.download('RW1S_events_start_end.csv')

##Merge precipitation and runoff data on minute interval

In [ ]:
time_index_n = pd.date_range(
    start='2016-10-01 00:00:00',
    end='2024-09-30 00:00:00',
    freq='1min'
)

df_time_n = pd.DataFrame({'Date': time_index_n})

In [ ]:
df_master_n = df_time_n.merge(
    df_q_n,
    on='Date',
    how='left'
)

In [ ]:
df_master_n = df_master_n.merge(
    df_p_n[['Date', 'Rainfall_mm']],
    on='Date',
    how='left'
)

Create columns with date and time only

In [ ]:
df_master_n['Date1'] = df_master_n['Date'].dt.date
df_master_n['Time'] = df_master_n['Date'].dt.time

##Separate events

In [ ]:
df_master_n = df_master_n.sort_values('Date')
events_time_n = events_time_n.sort_values('start_time')

In [ ]:
df_master_n['Date'] = pd.to_datetime(df_master_n['Date']) #garantir datetime completo

df_master_n['date'] = df_master_n['Date'].dt.date
df_master_n['is_midnight'] = (
    (df_master_n['Date'].dt.hour == 0) &
    (df_master_n['Date'].dt.minute == 0)
) #criar colunas auxiliares uma única vez

In [ ]:
dfs_events_n = []

for _, ev in events_time_n.iterrows():

    start = ev['start_time']
    end = ev['end_time']
    event_id = ev['Event']

        start_rain = start - pd.Timedelta(hours=6) #para considerar o início da chuva 6 horas antes do início do escoamento

    mask_event = (
        (df_master_n['Date'] >= start_rain) &
        (df_master_n['Date'] <= end)
    ) #novo intervalo do evento, considerando a precipitação de 6 horas

    df_ev_n = df_master_n.loc[mask_event].copy() #usar .copy() para evitar SettingWithCopy warnings depois

    df_ev_n.loc[df_ev_n['Date'] < start, 'Volume_mm'] = 0 #aqui ele vai zerar o escoamento antes da chuva começar: ERRADO!!

    days_event_n = pd.date_range(
        start_rain.date(),
        end.date(),
        freq='D'
    ).date #para pegar os volumes acumulados quando tem leitura a meia-noite

        mask_daily = (
        df_master_n['date'].isin(days_event_n) &
        df_master_n['is_midnight'] &
        df_master_n['Rainfall_mm'].notna()
    )
    df_daily_n = df_master_n.loc[mask_daily]

    df_event_full_n = (
        pd.concat([df_ev_n, df_daily_n])
        .drop_duplicates(subset='Date')
        .sort_values('Date')
    ) #aqui vai unir todos os eventos e ordená-los

    df_event_full_n = df_event_full_n.copy()
    df_event_full_n['Event'] = event_id

    dfs_events_n.append(df_event_full_n)

In [ ]:
df_master_events_ext_n = pd.concat(dfs_events_n, ignore_index=True)

In [ ]:
#Save
df_master_events_ext_n.to_csv('RWN1_events_extended.csv', index=True)

#Download
files.download('RWN1_events_extended.csv')

##Sum event variables

In [ ]:
event_sums_n = (
    df_master_events_ext_n
    .groupby('Event', as_index=False)
    .agg(
        Volume_event_mm=('Volume_mm', 'sum'),
        Rainfall_event_mm=('Rainfall_mm', 'sum')
    )
)

In [ ]:
event_sums_n[
    (event_sums['Rainfall_event_mm'] == 0) |
    (event_sums['Volume_event_mm'] == 0)
]

In [ ]:
minutes_per_event_n = (
    df_master_events_ext_n
    .groupby('Event')['Date']
    .count()
    .reset_index(name='n_minutes')
)

event_sums_n = event_sums_n.merge(minutes_per_event_n, on='Event')

In [ ]:
event_sums_n

In [ ]:
#Save
event_sums_n.to_csv('RWN1_sums.csv', index=True)

#Download
files.download('RWN1_sums.csv')

##RW SOUTH SITE

##Get rainfall data

In [ ]:
id = '1AJzpMK6QYbVPqdcseUkIfdKCwBDHttrcKxfWti2iAec' #id da planilha do google sheets - retirado do url
spreadsheet = gc.open_by_key(id) #para abrir spreadsheet pela chave
sheet = spreadsheet.worksheet("Precipitation") #nome da planilha a ser trabalhada
rows = sheet.get_all_values()
df_p_s = pd.DataFrame(rows[1:]) #transforma em DataFrame, a partir da linha 1 (linha 0 = cabeçalho)
df_p_s.columns = rows[0] #para transformar a primeira linha

In [ ]:
df_p_s.info()

In [ ]:
df_p_s['Rainfall_mm'] = pd.to_numeric(df_p_s['Rainfall_mm'], errors='coerce') #transformar object em float
df_p_s['Rainfall_mm'].fillna(0, inplace=True) #preenche NA com 0
df_p_s['Date'] = pd.to_datetime(df_p_s['Date'], format="%m/%d/%Y %H:%M:%S") #converte string para datetime

In [ ]:
df_p_s.info()

##Get runoff data

In [ ]:
id = '1AJzpMK6QYbVPqdcseUkIfdKCwBDHttrcKxfWti2iAec' #id da planilha do google sheets - retirado do url
spreadsheet = gc.open_by_key(id) #para abrir spreadsheet pela chave
sheet = spreadsheet.worksheet("Runoff") #nome da planilha a ser trabalhada
rows = sheet.get_all_values()
df_q_s = pd.DataFrame(rows[1:]) #transforma em DataFrame, a partir da linha 1 (linha 0 = cabeçalho)
df_q_s.columns = rows[0] #para transformar a primeira linha

In [ ]:
df_q_s.info()

In [ ]:
df_q_s['Volume_mm'] = pd.to_numeric(df_q_s['Volume_mm'], errors='coerce') #transformar object em float
df_q_s['Volume_mm'].fillna(0, inplace=True) #preenche NA com 0
df_q_s['Event'] = pd.to_numeric(df_q_s['Event'], errors='coerce') #transformar object em float
df_q_s['Event'].fillna(0, inplace=True) #preenche NA com 0
df_q_s['Date'] = pd.to_datetime(df_q_s['Date'], format="%m/%d/%Y %H:%M:%S") #converte string para datetime

In [ ]:
df_q_s.info()

##Get start and end of each event

In [ ]:
events_time_s = (
    df_q_s
    .groupby('Event')['Date']
    .agg(start_time='min', end_time='max')
    .reset_index()
)

In [ ]:
events_time_s = events_time_s[events_time_s['Event'] != 0]

In [ ]:
#Save
events_time_s.to_csv('RW1S_events_start_end.csv', index=True)

#Download
files.download('RW1S_events_start_end.csv')

##Merge precipitation and runoff data on minute interval

In [ ]:
time_index_s = pd.date_range(
    start='2016-10-01 00:00:00',
    end='2024-09-30 00:00:00',
    freq='1min'
)

df_time_s = pd.DataFrame({'Date': time_index_s})

In [ ]:
df_master_s = df_time_s.merge(
    df_q_s,
    on='Date',
    how='left'
)

In [ ]:
df_master_s = df_master_s.merge(
    df_p_s[['Date', 'Rainfall_mm']],
    on='Date',
    how='left'
)

Create columns with date and time only

In [ ]:
df_master_s['Date1'] = df_master_s['Date'].dt.date
df_master_s['Time'] = df_master_s['Date'].dt.time

##Separate events

In [ ]:
df_master_s = df_master_s.sort_values('Date')
events_time_s = events_time_s.sort_values('start_time')

In [ ]:
df_master_s['Date'] = pd.to_datetime(df_master_s['Date'])

df_master_s['date'] = df_master_s['Date'].dt.date
df_master_s['is_midnight'] = (
    (df_master_s['Date'].dt.hour == 0) &
    (df_master_s['Date'].dt.minute == 0)
)

In [ ]:
dfs_events_s = []

for _, ev in events_time_s.iterrows():

    start = ev['start_time']
    end = ev['end_time']
    event_id = ev['Event']

     start_rain = start - pd.Timedelta(hours=6)

     mask_event = (
        (df_master_s['Date'] >= start_rain) &
        (df_master_s['Date'] <= end)
    )

    df_ev_s = df_master_s.loc[mask_event].copy()


    df_ev_s.loc[df_ev_s['Date'] < start, 'Volume_mm'] = 0 #aqui está errado porque ele zera o escoamento


    days_event_s = pd.date_range(
        start_rain.date(),
        end.date(),
        freq='D'
    ).date


    mask_daily = (
        df_master_s['date'].isin(days_event_s) &
        df_master_s['is_midnight'] &
        df_master_s['Rainfall_mm'].notna()
    )
    df_daily_s = df_master_s.loc[mask_daily]


    df_event_full_s = (
        pd.concat([df_ev_s, df_daily_s])
        .drop_duplicates(subset='Date')
        .sort_values('Date')
    )

    df_event_full_s = df_event_full_s.copy()
    df_event_full_s['Event'] = event_id

    dfs_events_s.append(df_event_full_s)

In [ ]:
df_master_events_ext_s = pd.concat(dfs_events_s, ignore_index=True)

In [ ]:
#Save
df_master_events_ext_s.to_csv('RW1S_events_extended.csv', index=True)

#Download
files.download('RW1S_events_extended.csv')

##Sum event variables

In [ ]:
event_sums_s = (
    df_master_events_ext_s
    .groupby('Event', as_index=False)
    .agg(
        Volume_event_mm=('Volume_mm', 'sum'),
        Rainfall_event_mm=('Rainfall_mm', 'sum')
    )
)

In [ ]:
event_sums_s[
    (event_sums_s['Rainfall_event_mm'] == 0) |
    (event_sums_s['Volume_event_mm'] == 0)
]

In [ ]:
minutes_per_event_s = (
    df_master_events_ext_s
    .groupby('Event')['Date']
    .count()
    .reset_index(name='n_minutes')
)

event_sums_s = event_sums_s.merge(minutes_per_event_s, on='Event')

In [ ]:
event_sums_s

In [ ]:
#Save
event_sums_s.to_csv('RW1S_sums.csv', index=True)

#Download
files.download('RW1S_sums.csv')

#MC Site

##Get rainfall data

In [ ]:
id = '1gXo0bOYZxrkakpnB8twwDeczVk4Qg5xMV9HtVop1dP4' #id da planilha do google sheets - retirado do url
spreadsheet = gc.open_by_key(id) #para abrir spreadsheet pela chave
sheet = spreadsheet.worksheet("Precipitation") #nome da planilha a ser trabalhada
rows = sheet.get_all_values()
df_p_mc = pd.DataFrame(rows[1:]) #transforma em DataFrame, a partir da linha 1 (linha 0 = cabeçalho)
df_p_mc.columns = rows[0] #para transformar a primeira linha

In [ ]:
df_p_mc.info()

In [ ]:
df_p_mc['Rainfall_mm'] = pd.to_numeric(df_p_mc['Rainfall_mm'], errors='coerce') #transformar object em float
df_p_mc['Rainfall_mm'].fillna(0, inplace=True) #preenche NA com 0
df_p_mc['Date'] = pd.to_datetime(df_p_mc['Date'], format="%m/%d/%Y %H:%M:%S") #converte string para datetime

In [ ]:
df_p_mc.info()

#Get start and end of each event

In [ ]:
id = '1gXo0bOYZxrkakpnB8twwDeczVk4Qg5xMV9HtVop1dP4' #id da planilha do google sheets - retirado do url
spreadsheet = gc.open_by_key(id) #para abrir spreadsheet pela chave
sheet = spreadsheet.worksheet("Runoff") #nome da planilha a ser trabalhada
rows = sheet.get_all_values()
df_q_mc = pd.DataFrame(rows[1:]) #transforma em DataFrame, a partir da linha 1 (linha 0 = cabeçalho)
df_q_mc.columns = rows[0] #para transformar a primeira linha

In [ ]:
df_q_mc.info()

In [ ]:
df_q_mc['Volume_mm'] = pd.to_numeric(df_q_mc['Volume_mm'], errors='coerce') #transformar object em float
df_q_mc['Volume_mm'].fillna(0, inplace=True) #preenche NA com 0
df_q_mc['Start_event'] = pd.to_datetime(df_q_mc['Start_event'], format="%m/%d/%Y %H:%M:%S") #converte string para datetime
df_q_mc['End_event'] = pd.to_datetime(df_q_mc['End_event'], format="%m/%d/%Y %H:%M:%S") #converte string para datetime

In [ ]:
df_q_mc.info()

##Sum rainfall during each event

In [ ]:
def soma_chuva_evento(df_p_mc, inicio, fim):
    mask = (df_p_mc['Date'] >= inicio) & (df_p_mc['Date'] <= fim)
    return df_p_mc.loc[mask, 'Rainfall_mm'].sum()

##Apply function to each event

In [ ]:
df_q_mc['chuva_total_mm'] = df_q_mc.apply(
    lambda row: soma_chuva_evento(
        df_p_mc,
        row['Start_event'],
        row['End_event']
    ),
    axis=1
)

In [ ]:
#Save
df_q_mc.to_csv('MC1_sums.csv', index=True)

#Download
files.download('MC1_sums.csv')